# Recipe Recommender Assignment

In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1687670111475_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Basics").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Imports

In [4]:
# Run this everytime you create a new spark instance. 

spark.sparkContext.install_pypi_package("plotly==5.5.0")
spark.sparkContext.install_pypi_package("pandas==0.25.1")
spark.sparkContext.install_pypi_package("numpy==1.14.5")
spark.sparkContext.install_pypi_package("matplotlib==3.1.1")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/b3/f4/23d4a698db9fe772f7fdf40ac17b743c4b0d80274732c59db5bd45acb3be/plotly-5.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e7/b0/c23bd61e1b32c9b96fbca996c87784e196a812da8d621d8d04851f6c8181/tenacity-8.2.2-py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/3f/e7/7f24ef402a5766c677683e313c5595137d754cb9eb1c99627803280e79d5/numpy-1.14.5-cp37-cp37m-manylinux1_x86_64.whl
  Found existing installation: numpy 1.20.0
    Not uninstalling numpy at /usr/local/lib64/python3.7/site-packages, outside environment /tmp/1687675998234-0

  Using cached https://files

In [5]:
spark.sparkContext.install_pypi_package("s3fs==2023.6.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Could not find a version that satisfies the requirement s3fs==2023.6.0 (from versions: 0.0.1, 0.0.2, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.2.0, 0.2.1, 0.2.2, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.4.0, 0.4.1, 0.4.2, 0.5.0, 0.5.1, 0.5.2, 0.6.0, 2021.4.0, 2021.5.0, 2021.6.0, 2021.6.1, 2021.7.0, 2021.8.0, 2021.8.1, 2021.9.0, 2021.10.0, 2021.10.1, 2021.11.0, 2021.11.1, 2022.1.0, 2022.2.0, 2022.3.0, 2022.5.0, 2022.7.0, 2022.7.1, 2022.8.0, 2022.8.1, 2022.8.2, 2022.10.0, 2022.11.0, 2023.1.0)
No matching distribution found for s3fs==2023.6.0
You are using pip version 9.0.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [6]:
# import necessary libraries 
import pandas as pd
import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType, ArrayType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the data

In [9]:
train_ratings_df = spark.read.parquet('s3://aws-emr-resources-757926750210-us-east-1/train_interaction_level_df.parquet') # Replace the given path with the path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
train_ratings_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-------------------+--------------------+-----+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
|user_id|recipe_id|rating|              review|        review_date|                name|   id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|year_of_review|
+-------+---------+------+--------------------+-------------------+--------------------+-----+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
| 180507|     5084|     4|Very similar to t...|2005-10-04 00:00:00|granny s chicken ...| 5084|      0|          1534|1999-12-01 00:00:00|['15-minutes-or-l...|[2

In [11]:
test_ratings_all_df  = spark.read.parquet('s3://aws-emr-resources-757926750210-us-east-1/test_interaction_level_df.parquet') # Replace the given path with the path for your file


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
test_ratings_all_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-------------------+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
|user_id|recipe_id|rating|              review|        review_date|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|year_of_review|
+-------+---------+------+--------------------+-------------------+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
| 222564|   301798|     5|DIVINE! I didn&#0...|2014-05-11 00:00:00|bow tie pasta wit...|301798|     35|        324390|2008-05-01 00:00:00|['60-minutes-or-l..

In [13]:
raw_recipes_df = spark.read.csv("s3a://raw-recipes-clean-upgrad/RAW_recipes_cleaned.csv") # Replace the given path with the path for your file
                            

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
raw_recipes_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|                 _c0|   _c1|    _c2|           _c3|       _c4|                 _c5|                 _c6|    _c7|                 _c8|                 _c9|                _c10|         _c11|
+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|                name|    id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
|arriba   baked wi...|137739|     55|         47892|2005-09-16|['60-minutes-or-l...|[51.5, 0.0, 13.0,...|     11|['make a choice a...|autumn is my favo...|['winter squash',...|            7|
|a bit different  ...| 31490|     30|        

In [15]:
raw_recipes_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11']

In [16]:
# Count the total number of ratings in the dataset
numerator = train_ratings_df.select("rating").count()

# Count the number of distinct userIds and distinct recipe_Ids
num_users =train_ratings_df.select("user_id").distinct().count() # find the number of unique users in the training data. The output must be an integer
num_recipes =train_ratings_df.select("recipe_id").distinct().count() # find the number of unique recipes in the training data. The output must be an integer

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_recipes

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The training dataframe is ", "%.7f" % sparsity + "% empty.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The training dataframe is  99.9829264% empty.

## Functions

In [17]:
def manual_recommendation_check (user_id):
  '''
  Given a user ID form the test dataset, this function will return the names of the recipes recommended to the user. 

  Initialize this function after the all_recs data frame is calculated. 

  Input user_id of a user from the test set as an integer.  
  Prints the names of recipes recommended to this user. 
  Returns nothing. 
  '''
  recs_user = all_recs_als[all_recs_als.user == user_id]
  recs_user_list = list(recs_user.item.values)
  recs_user_list = [x.item() for x in recs_user_list]
  display((raw_recipes_df.filter(F.col('id').isin(recs_user_list))
               .select("name")
               .collect()
               ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Model

#### Task 02 - Collaborative Filtering Model

Add the argument details in the algorithm initialization below to build the ALS model.

In [18]:
# Create ALS model
als = ALS(userCol="user_id" , # add the name of the column for users  
          itemCol="recipe_id", # add the name of the column for recipes
          ratingCol="rating", # add the name of the column for ratings 
          nonnegative = True, 
          implicitPrefs = False, 
          coldStartStrategy="drop"
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Training

In [19]:
#Fit the model to the 'train' dataset
model = als.fit(train_ratings_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Prediction 

In [20]:
# use the model to create predictions for test data
test_predictions_unseen =model.transform(test_ratings_all_df)  # add a statment to tranform the test data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
test_predictions_unseen.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-------------------+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+----------+
|user_id|recipe_id|rating|              review|        review_date|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|year_of_review|prediction|
+-------+---------+------+--------------------+-------------------+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+----------+
|  56680|   229831|     5|Perfection  made ...|2013-11-03 00:00:00|tavern beer batte...|229831|     15|        339260|2007-0

In [22]:
assert test_predictions_unseen.select(F.col("prediction"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
test_predictions_unseen.select(F.col("user_id"),
                               F.col("recipe_id"),
                               F.col("rating"),
                               F.col("prediction")
                              ).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+----------+
|user_id|recipe_id|rating|prediction|
+-------+---------+------+----------+
|  56680|   229831|     5| 4.5213604|
|  65308|   139797|     5| 3.7124462|
|  65308|    51088|     0|  4.332805|
|  65308|   354072|     5| 4.0239716|
| 657535|    37470|     1| 1.4968789|
+-------+---------+------+----------+
only showing top 5 rows

In [24]:
# Each user in the test set must have 10 predictions. 
# Use ALS model to get these predictions. 
# You can use the recommendForAllUsers() method. 

recommendations = model.recommendForAllUsers(10) # complete the code 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
assert len(dict(recommendations.select(F.col("recommendations")).collect()[0][0])) == 10

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Transfroming the results by exploding the recommendations column

recommendations = (recommendations.select(F.col("user_id"),
                                          F.posexplode(F.col("recommendations")).alias("pos", "item")) 
                                  .select(F.col("user_id"),
                                          F.col("pos"), 
                                          F.col("item.recipe_id").alias("recomended_recipe_id"), 
                                          F.col("item.rating").alias("predicted_rating")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Evaluation

### Task 04 - Model Evaluation

#### 1. RMSE

Add the argument details to the evaluator function below to calculate the RMSE score of the ALS model. 

In [27]:
# Define evaluator as RMSE and print RMSE value
evaluator_seen = RegressionEvaluator(metricName="rmse", 
                                     labelCol="rating" ,  # add the name of the column that has the ratings 
                                     predictionCol="prediction" # add the name of the column that has the predicted ratings. 
                                     ) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
def computeRmse(model, data):
    """
    Compute RMSE (Root mean Squared Error).
    """
    prediction = model.transform(data)
    rmse = evaluator_seen.evaluate(prediction)
    print("Root-mean-square error = " + str(rmse)) 
    return rmse         

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Use the evaluator to find the RMSE on the test set. 
validationRmse = computeRmse(model, test_ratings_all_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root-mean-square error = 1.411216458241111

#### Rank based Metrics

We will use the lenskit library to calculate the ranking-based matrics. The lenskit library is available in pandas only so we need to convert the data frames from PySpark dataframes to Pandas dataframes. 

In [30]:
all_recs_als = recommendations.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
all_recs_als

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        user_id  pos  recomended_recipe_id  predicted_rating
0          5427    0                346418          5.821311
1          5427    1                197397          5.633894
2          5427    2                142084          5.632503
3          5427    3                173484          5.601570
4          5427    4                247044          5.585267
...         ...  ...                   ...               ...
233615  1881531    5                236802          5.652461
233616  1881531    6                203117          5.614207
233617  1881531    7                243882          5.612392
233618  1881531    8                184566          5.608384
233619  1881531    9                333022          5.599547

[233620 rows x 4 columns]

In [32]:
# Rename the columns of to eunsre that they match the columns names as in the cell below. 

column_names ={"user_id":"user","pos":"rank","recomended_recipe_id":"item","predicted_rating":"score"} # create a dictionary with current column names as keys and the intended column names as values. 
all_recs_als = all_recs_als.rename(columns=column_names)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
all_recs_als

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           user  rank    item     score
0          5427     0  346418  5.821311
1          5427     1  197397  5.633894
2          5427     2  142084  5.632503
3          5427     3  173484  5.601570
4          5427     4  247044  5.585267
...         ...   ...     ...       ...
233615  1881531     5  236802  5.652461
233616  1881531     6  203117  5.614207
233617  1881531     7  243882  5.612392
233618  1881531     8  184566  5.608384
233619  1881531     9  333022  5.599547

[233620 rows x 4 columns]

In [34]:
# Adding a column to make sure the we know these recommendations are from the ALS algorithm. 

all_recs_als["algorithm"] = "ALS"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
all_recs_als

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           user  rank    item     score algorithm
0          5427     0  346418  5.821311       ALS
1          5427     1  197397  5.633894       ALS
2          5427     2  142084  5.632503       ALS
3          5427     3  173484  5.601570       ALS
4          5427     4  247044  5.585267       ALS
...         ...   ...     ...       ...       ...
233615  1881531     5  236802  5.652461       ALS
233616  1881531     6  203117  5.614207       ALS
233617  1881531     7  243882  5.612392       ALS
233618  1881531     8  184566  5.608384       ALS
233619  1881531     9  333022  5.599547       ALS

[233620 rows x 5 columns]

In [36]:
# Convert the test dataset to pandas and ensure that it has the same column names as shown in the cell below. 
# Also, note there are only three columns in the rest data. You have to ensure your test data looks identical. 

test_data = (test_ratings_all_df.toPandas())# add code to select the necessary columns and change the name of the columns. 
test_data=test_data.rename(columns={"user_id":"user","recipe_id":"item","rating":"rating"})
test_data=test_data[["user","item","rating"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
test_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

             user    item  rating
0          222564  301798       5
1         2894019   31988       5
2         1271506   31988       5
3          647495   31988       5
4           85018   31988       5
...           ...     ...     ...
18679     1981063   50684       5
18680     1765320   50684       5
18681  1801745624   50684       5
18682     2389315  457576       5
18683      440735  457576       4

[18684 rows x 3 columns]

In [38]:
# # code to calculate the necessary metrics 
# # the code below uses lenskit library to evaluate the rank metrics

# rla = topn.RecListAnalysis()
# rla.add_metric(topn.recip_rank)
# rla.add_metric(topn.ndcg)
# rla.add_metric(topn.dcg)
# results = rla.compute(all_recs_als, test_data)
# results.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Manual Prediction Checking

1. Why are ndcg and dcg nulls? EG: user 28170

In [39]:
test_data[test_data.user == 28170]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Empty DataFrame
Columns: [user, item, rating]
Index: []

In [40]:
all_recs_als[all_recs_als.user == 28170]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        user  rank    item     score algorithm
92440  28170     0  272390  5.915882       ALS
92441  28170     1   98791  5.877940       ALS
92442  28170     2  302808  5.865075       ALS
92443  28170     3  178137  5.796546       ALS
92444  28170     4   36802  5.728985       ALS
92445  28170     5  257533  5.715683       ALS
92446  28170     6  322821  5.695846       ALS
92447  28170     7  146694  5.688215       ALS
92448  28170     8   16768  5.676943       ALS
92449  28170     9  331217  5.674032       ALS

User ```28170``` does not appear in the test set. Hence cannot be evaluated. 

2. Why are all metrics 0 for specific users?

In [41]:
test_data[test_data.user == 56680]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        user    item  rating
17647  56680  229831       5

In [42]:
all_recs_als[all_recs_als.user == 56680]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      user  rank    item     score algorithm
750  56680     0   85238  6.313882       ALS
751  56680     1  401773  6.126190       ALS
752  56680     2  339583  6.122345       ALS
753  56680     3  410035  6.122345       ALS
754  56680     4   46802  6.115890       ALS
755  56680     5  173484  6.099307       ALS
756  56680     6  336380  6.013658       ALS
757  56680     7   75776  5.997461       ALS
758  56680     8  210216  5.997461       ALS
759  56680     9  379283  5.993109       ALS

User-recipe combination does not appear in the recommendations set. 